<h2>Introduction</h2>

Here you will find a step by step guide to downloading, configuring, and running the Einstein Toolkit. You may use this tutorial on a workstation or laptop, or on a supported cluster. Configuring the Einstein Toolkit on an unsupported cluster is beyond the scope of this tutorial. If you find something that does not work, please feel free to mail users@einsteintoolkit.org.

<h2>Prerequisites</h2>
When using the Einstein Toolkit on a laptop or workstation you will want a number of packages installed in order to download, compile and use the Einstein Toolkit components. If this is a machine which you control (i.e. you have root), you can install using one of the recipes that follow:

On Mac, please first 
- Install [Xcode](https://itunes.apple.com/us/app/xcode/id497799835) from the Apple [App Store](https://itunes.apple.com/us/app/xcode/id497799835). In *addition* agree to Xcode license and install the Xcode Command Line Tools in Terminal 
```bash
sudo xcodebuild -license
sudo xcode-select --install
```
- when using MacPorts
  - install MacPorts for your version of the Mac operating system, if you have not already installed it (https://www.macports.org/install.php). 
  - Next, please install the following packages, using the commands:
```bash
sudo port -N install pkgconfig gcc9 openmpi-gcc9 fftw-3 gsl jpeg zlib hdf5 +fortran +gfortran openssl subversion ld64
sudo port select mpi openmpi-gcc9-fortran
sudo port select gcc mp-gcc9
```
- when using HomeBrew
  - install HomeBrew for your version of the Mac operating system, if you have not already installed it (https://brew.sh/). 
  - Next, please install the following packages, using the commands:
```bash
brew install fftw gcc gsl hdf5 hwloc jpeg open-mpi pkg-config subversion
```

On Debian/Ubuntu/Mint use this command (the strange syntax is to suport all three of them):
```bash
$(sudo -l sudo) su -c 'apt-get install -y subversion gcc git numactl libgsl-dev libpapi-dev python libhwloc-dev libudev-dev make libopenmpi-dev libhdf5-openmpi-dev libfftw3-dev libssl-dev liblapack-dev g++ curl gfortran patch pkg-config libhdf5-dev libjpeg-turbo?-dev'
```

On Fedora use this command:
```bash
sudo dnf install -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which papi-devel python2 hwloc-devel openmpi-devel hdf5-openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran findutils hdf5-devel fftw-devel patch gsl-devel pkgconfig
module load mpi/openmpi-x86_64
```
You will have to repeat the `module load` command once in each new shell each time you would like to compile or run the code. You may have to log out and back in for the module command to become visible.

On Centos use this command:
```bash
su -c 'yum install -y epel-release'
su -c 'yum install -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which papi-devel hwloc-devel openmpi-devel hdf5-openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran hdf5-devel fftw-devel patch gsl-devel'
module load mpi/openmpi-x86_64
```
You will have to repeat the `module load` command once in each new shell each time you would like to compile or run the code. You may have to log out and back in for the module command to become visible.

On OpenSuse use this command:
```bash
sudo zypper install -y curl gcc git lapack-devel make subversion gcc-c++ which papi-devel hwloc-devel openmpi-devel libopenssl-devel libnuma-devel gcc-fortran hdf5-devel libfftw3-3 patch gsl-devel pkg-config
sudo mpi-selector --set  $(mpi-selector --list | head -n1)
```
You will only have to execute the `mpi-selector` once, after that log out and back in to make the `mpirun` and `mpicc` commands visible without which Cactus will compile very slowly and fail to run.

On Windows 10 please install the Ubuntu Linux subsystem, then follow the instructions for a Ubuntu system. [These](https://docs.microsoft.com/en-us/windows/wsl/install-win10) are Microsoft's official instructions on how to do so, [Ubuntu](https://ubuntu.com/wsl#install-ubuntu-on-wsl) provides an alternative version. You may also want to install native ssh client like [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/) and an X11 server like [VcXsrv](https://sourceforge.net/projects/vcxsrv/), [XMing](https://sourceforge.net/projects/xming/) or an all-in-one solution for X11 server and ssh client like [MobaXterm](https://mobaxterm.mobatek.net/).

## Notebook setup
This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document, as a jupyter notebook that you can download and also in your own docker container using `ndslabs/jupyter-et`. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)
# Some versions of OpenMPI prevent oversubscribing cpus, which may happen if simfactory's
# number of cores detection is imperfect.
# OpenMPI by default pins MPI ranks to specific cores, which causes issues on shared
# system like the tutorial cluster.
# OpenMPI contains a bug affecting MPI calls with large amounts of data on slow systems,
# which can lead to hangs (OpenMPI issue 6568).
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"] = "true"
os.environ["OMPI_MCA_hwloc_base_binding_policy"] = "none"
os.environ["OMPI_MCA_btl_vader_single_copy_mechanism"] = "none"

<h2>Download</h2>

A script called GetComponents is used to fetch the components of the Einstein Toolkit. GetComponents serves as convenient wrapper around lower level tools like git and svn to download the codes that make up the Einstein toolkit from their individual repositories. You may download and make it executable as follows:

<b>Note:</b> By default, the cells in this notebook are Python commands. However, cells that start with <code>%%bash</code> are executed in a bash shell. If you wish to run these commands outside the notebook and in a bash shell, cut and paste only the part after the initial <code>%%bash</code>. 

In [ ]:
cd ~/

In [ ]:
%%bash
curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2020_05/GetComponents
chmod a+x GetComponents

GetComponents accepts a thorn list as an argument. To check out the needed components:

In [ ]:
%%bash
./GetComponents --parallel https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2020_05/einsteintoolkit.th

In [ ]:
cd ~/Cactus

<h2>Configure and build</h2>

The recommended way to compile the Einstein Toolkit is to use the Simulation Factory ("SimFactory").
<h3>Configuring SimFactory for your machine</h3>

The ET depends on various libraries, and needs to interact with machine-specific queueing systems and MPI implementations. As such, it needs to be configured for a given machine. For this, it uses SimFactory. Generally, configuring SimFactory means providing an optionlist, for specifying library locations and build options, a submit script for using the batch queueing system, and a runscript, for specifying how Cactus should be run, e.g. which mpirun command to use.

In [ ]:
%%bash
./simfactory/bin/sim setup-silent

After this step is complete you will find your machine's default setup under ./simfactory/mdb/machines/&lt;hostname &gt;.ini
You can edit some of these settings freely, such as "description", "basedir" etc. Some entry edits could result in simulation start-up warnings and/or errors such as "ppn" (processor-per-node meaning number of cores on your machine), "num-threads" (number of threads per core) so such edits must be done with some care.
### Issues with GCC version 10
`gfortran` and `gcc` version 10 fail to compile the Toolkit due to stricter enforcement of language standards. To restore the old behaviour please edit the optionlist file ./simfactory/mdb/optionlists/&lt;optionlist&gt;.cfg mentioned in the machine defintion file ./simfactory/mdb/machines/&lt;hostname &gt;.ini by adding `-fcommon` to `CFLAGS`, `CXXFLAGS` and `LDFLAGS` and `-fallow-argument-mismatch` to `F90FLAGS`.

## Cactus reduction interface

Cactus provides functionality to compute "reductions" of the value on the grid,
similar to the reduce operations defined by MPI. These operations include
finding the maximum or minimum value of a grid variable, or computing sums or
norms of grid variable data.

The exact set of operations is not defined by Cactus but instead determined by a
combination which driver thorn (PUGH or Carpet) and reduction thorn (PUGHReduce 
and LocalReduce or CarpetReduce) are active. Among the moe commonly used 
reductions provided by both drivers are:

<table>
    <tr><th>name (PUGH / Carpet)</th><th>description</th></tr>
    <tr><td><code>min</code> / <code>minimum</code></td><td>minimum value on the grid</td></tr>
    <tr><td><code>max</code> / <code>maximum</code></td><td>maximum value on the grid</td></tr>
    <tr><td><code>norm_inf</code></td><td>infinity norm the grid, i.e. $\max |\psi|$</td></tr>
    <tr><td><code>sum</code></td><td>volume weighted sum on the grid, i.e. Riemann sum</td></tr>
    <tr><td><code>norm1</code></td><td>volume weighted $L_1$ norm on the grid</td></tr>
    <tr><td><code>norm2</code></td><td>volume weighted $L_2$ norm on the grid</td></tr>
    <tr><td><code>average</code></td><td>volume weighted average on the grid</td></tr>
</table>

Note that Cactus does not define equivalents of MPI's `min_loc` and `max_loc`
operations.

The volume weighted sum is defined as
$\sum \psi \equiv \left(\sum_i \psi_i \, \Delta V_i\right) / \Delta V$ where 
$i$ runs over all the grid points, $\Delta_i$ is the volume of cell $i$ that
contributes to the sum and $\Delta V$ is the volume of a reference cell on the
coarset refinement level. For mesh refined simulations $\Delta V_i$ depends on
whether a given cell is (partially) overlapped by a cell on a finer refinement
level.

Cactus defines `sum` reductions such that

$$
\texttt{sum}(\psi_i; t) \, \Delta V \rightarrow \int \psi(t, x) \, d^3x
$$

as resolution is increased, where $V$ is the (constant) simulation domain.

## Using reductions

Reduction can be used either during output as options in the
`IOBasic::reductions` parameters, or at runtime in the `CCTK_ReduceGridArays`
(PUGH) or `CCTK_Reduce` (Carpet) calls.

When used for ouptut the output thorns will compute the requested reductions for
all output variables and write the result to disk / screen computing scalar
quantities.

The programmatic functions `CCTK_ReduceGridArays` and `CCTK_Reduce` allow
application thorns to do the same for example to write custom output files or to
store reduction output in grid scalars for later use.

In this tutorial we will focus only on the `CCTK_Reduce` interface offered by
Carpet.

```C
int CCTK_ReductionHandle(const char *reduction_name);

int CCTK_Reduce(const cGH *GH,          /* Cactus grid hierarchy */
                int proc,               /* MPI rank to receive result, -1 for all */
                int operation_handle,   /* obtained from CCTK_ReductionHandle */
                int num_out_vals,       /* number of output values per grid array */
                int type_out_vals,      /* CCTK_VARIABLE_INT or CCTK_VARIABLE_REAL */
                void *out_vals,         /* point to array of size num_out_vals for results */
                int num_in_fields,      /* speciﬁes the number of input ﬁelds */
                ...);                   /* variable indices of grid variables to act on */
```

## Example use

This fragments shows how to compute the `sum` reduction of `GRHydro::dens` which
is, up to the normalization factor $\Delta V$ the rest mass
$M_0(t) = \int \rho_*(t,x) \, \sqrt{\gamma(t,x)} \, d^3x$.


### Code
```C
const int sum_handle = CCTK_ReductionHandle("sum");
if(sum_handle < 0)
  CCTK_VERROR("Could not obtain 'sum' reduction handle: %d", sum_handle);

const int varindex = CCTK_VarIndex("GRHydro::dens");
if(varindex < 0) {
  CCTK_VERROR("Could not obtain variable index for GRHydro::dens: %d",
              varindex);
}

CCTK_REAL rest_mass = 42.;
const int ierr = CCTK_Reduce(cctkGH, -1 /* proc */, sum_handle, 1 /* num_out_vals */,
                             CCTK_VARIABLE_COMPLEX, &rest_mass, 1 /* num_in_fields */,
                             varindex);
if(ierr < 0)
  CCTK_VERROR("Could not compute 'sum' reduction for GRHydro::dens: %d", ierr);

CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

### Schedule

```Perl
schedule restmass_computerestmass IN CCTK_ANALYSIS
{
  LANG: C
  OPTIONS: global
} "compute rest mass"
```

## Mesh refinement

Mesh refinement, in particular subcylcing in time, adds complications to
computing reductions (and any other "global" quantity.).

* a (`LOCAL`) scheduled function is called once per grid patch
* different refinement levels exist at different times
* `sum` like reductions have to take relative size of grid cells into account
* during `EVOL` refinement levels are updated coarse-to-fine

### Modes

Carpet introduces the concept of "modes" to add a concept of acces levels to the
Cactus schedule:

<dl>
    <dt><code>LOCAL</code></dt><dd>called once per component, has access to
    poisition dependent data on the grid</dd>
    <dt><code>LEVEL</code></dt><dd>called once per refinement level,
    <code>SYNC</code> happens here, logically boundary conditions are speficied
    in this mode, the schedule is traversed in this mode. </dd>
    <dt><code>GLOBAL</code></dt><dd>called once per time step, has access to
    grid scalars and grid arrays only</dd>
</dl>

`CCTK_Reduce` can be called in either `LEVEL` or `GLOBAL` mode and will return
a reduction over all values on a *single* refinement level or *all* refinement
levels.

### Interpolation in time

Since `GLOBAL` routines are called for every since timestep, and not just every
"coarse" timestep, a reduction maybe called at times that do not correspond to a
time for which a coarse refinement level exists. In the case CarpetReduce will
interpolate grid data in time to provide coarse grid data at the required time.

### Cell volumes

Naively computing the sum of values on the grid once multiple refinement levels
exist is not very useful since such an operation does not correspond to any
continuum functional of the grid variables. Carpet this weighs each contribution
by the relative cell volume, producing an approximation of the Riemann sum. Since
Carpet does now know the physical volume that a cell corresponds to, a relative
cell volume normalized by the volume of the cells on the coarsest grid is used.

### `EVOL` vs `ANALYSIS`

01234567890123456789012345678901234567890123456789012345678901234567890123456789

Refinement levels are traversed coarse to fine, so during `EVOL` Carpet takes a
step on the coarsest level first, then a step with half the stepsize on the next
finest level, etc. meaning that data one coarse levels is *ahead* in time of the
fine levels. In `ANALYSIS` the fine refinement levels have caught up with the
coarse ones, so all refinement are at the *same* time.

In `EVOL` Carpet will call `GLOBAL` scheduled functions along with `LOCAL`
routines on the coarset level (first) so that XXX.

In `ANALYSIS` Carpet will call `GLOBAL` scheduled functiosn along with `LOCAL`
routines on the finest level (last) so that a reduction over a just computed
analysis quantity is possible.

## Sketch of schedule traversal using 2 refinement levels

```Perl
schedule take_step IN EVOL
{
    LANG: C
    OPTIONS: LOCAL
} "update to next time"

schedule compute_average IN EVOL AFTER take_step
{
    LANG: C
    OPTIONS: GLOBAL
} "compute average"

schedule compute_energy IN ANALYSIS
{
    LANG: C
    OPTIONS: GLOBAL
} "compute energy density"


schedule compute_total_energy IN ANALYSIS AFTER compute_energy
{
    LANG: C
    OPTIONS: GLOBAL
} "compute total energy"

```



<table cellspacing="0" border="0">
	<colgroup span="4" width="85"></colgroup>
	<colgroup width="123"></colgroup>
	<colgroup width="143"></colgroup>
	<colgroup width="230"></colgroup>
	<colgroup span="6" width="34"></colgroup>
	<tr>
		<td height="17" align="left" bgcolor="#B3B3B3"><b>iteration</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>cctk_time</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>reflevel</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>mode</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>bin</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>function</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>description</b></td>
		<td colspan=3 align="left" bgcolor="#999999"><b>reflevel 0</b></td>
		<td colspan=3 align="left" bgcolor="#CCCCCC"><b>reflevel 1</b></td>
		</tr>
	<tr>
		<td height="17" align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="left" bgcolor="#BABABA"><br></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FFFF00">global</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute_average</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute average</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 0.50</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FF00FF">OutputGH</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">output scalars at t=0.50</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#CCCCCC">restrict fine to coarse</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 0 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#FFFF00">compute_energy</td>
		<td align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">output scalars at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
</table>


<h2>Plotting the Output</h2>

The following commands will generate a simple line plot of the data. They will work in a python script as easily as they do in the notebook (just remove the "%matplotlib inline" directive).

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%%bash
# find out the directory that Cactus stored its output in
./simfactory/bin/sim get-output-dir tov_ET

In [ ]:
# store output directory location for later use
# in ipython you can also use this:
# outdir = ! ./simfactory/bin/sim get-output-dir tov_ET
# outdir = outdir[0]
import os
outdir = os.popen("./simfactory/bin/sim get-output-dir tov_ET").read().rstrip("\n")

print ("Output is written to", outdir)

Numpy has a routine called genfromtxt() which is an extremely efficient reader of textual arrays of floating point numbers. This is well-suited to Cactus .asc files.

In [ ]:
# format of the path: SIMULATION-NAME/output-NNNN/PARFILE-NAME
lin_data = np.genfromtxt(outdir+"/tov_ET/hydrobase-rho.maximum.asc")

This is all you need to do to plot the data once you've loaded it. Note, this uses Python array notation to grab columns 1 and 2 of the data file.

In [ ]:
plt.plot(lin_data[:,1],lin_data[:,2]/lin_data[0,2], label="central density")
plt.xlabel(r'$t$ [$M_{\odot}$]');
plt.ylabel(r'$\rho_c / \rho_c(0)$');
plt.legend();

In [ ]:
# create small dataset to show what plot should look like
def sparsify(lin_data, sparsity):
    # drop unwanted datapoint
    sparse_data = lin_data[::sparsity,:]
    
    # compute min, max of dataset then difference to minimum and quantize to 8 bit precisison
    minval = np.amin(sparse_data[:,2])
    maxval = np.amax(sparse_data[:,2])
    diff = sparse_data[:,2] - minval
    quant_diff = np.minimum(np.maximum(np.round(diff / (maxval - minval) * 255.5), 0), 255).astype('int')

    # timesteps are equidistant and start at 0 so we only need the stepsize
    delta_t = sparse_data[1,1] - sparse_data[0,1]

    # string rep of 8bit differences
    quant_diff_s = ""
    for i in quant_diff: quant_diff_s += "%02x" % i
    
    print ('"%s", %.3e, %.3e, %.3e' % (quant_diff_s, minval, maxval, delta_t))

# create a low fidelity representation of every 10th datapoint and output all data a string
sparsify(lin_data, 10)    

In [ ]:
# this cell shows the expected plot using previously stored data

# reconstruct plot data from saved strings
(quant_diff_s, minval, maxval, delta_t) = \
  ("ff8baee2e5d2ac70320c0007182c404f5b656f7b8897a8bbcddde8eeede8ddcfc0b0a29589817b777473757a8189929ca6b0bac4cbd0d3d4d4d2cfcbc7c2bdb8b4b0adaaa9a8a9abaeb3b8bcc1c5c8cccf",
   1.235e-03, 1.280e-03, 5.000e+00)
quant_diff = np.array(bytearray.fromhex(quant_diff_s))
rec_vals = quant_diff / 255. * (maxval- minval) + minval
rec_time = np.arange(0,len(quant_diff)) * delta_t

# plot them, including your results if you have them
plt.plot(rec_time, rec_vals/rec_vals[0],
         label="central density (stored values)")
try: plt.plot(lin_data[:,1],lin_data[:,2]/lin_data[0,2], label="central density (your results)")
except: pass
plt.xlabel(r'$t$ [$M_{\odot}$]');
plt.ylabel(r'$\rho_c / \rho_c(0)$');
plt.legend(loc='lower right');

Running the cell above will produce a plot of the expected results as well as your own results.
![Central density(stored value)](https://github.com/nds-org/jupyter-et/raw/master/data/tov_ET.png)